In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# DeprecationWarning: executable_path has been deprecated, please pass in a Service object
# 에러 발생으로 !pip install webdriver-manager 설치 후 아래 함수로 driver 실행
def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [2]:
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
import pandas as pd
import numpy as np

# 유튜브 플레이리스트 크롤링
- 감정에 따라 검색어를 지정해서 플레이리스트를 수집하기엔 일일히 검색어를 입력해야하고 다양하게 수집이 불가할 거 같아
- 플레이리스트 채널 구독자순위가 높은 채널링크에서 모든 영상을 수집하기로 함

### 채널 크롤링
- 1. 나의 플레이리스트 NAPLY 'https://www.youtube.com/channel/UC2kPuCfWPDZhRg60gOdOlKg/videos' : 42
- 2. 찐막 JJINMAK 'https://www.youtube.com/channel/UCxGyH7vzYkZwFseTUa3yZpQ/videos' : 114
- 3. 감성 저장소 'https://www.youtube.com/channel/UCTieDwi-vnwxBwJGruCWvTA/videos' : 178
- 4. feel it 'https://www.youtube.com/channel/UCmtRph7zudk7YZBnow-HAbg/videos' : 37
- 5. 라디안트RADIANT 'https://www.youtube.com/channel/UCRqHQRQJZZ9jrnqFyRadTag/videos' : 97

In [85]:
driver = set_chrome_driver()



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\admin\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [96]:
driver.quit() # 크롤링 끝나면 꼭!!

In [80]:
def channel_video_crawl(url):
    # 크롬 웹브라우저 실행
    driver.get(url)
    
    time.sleep(3)
    scroll = 7
    while scroll: #end키 눌러서 스크롤내리기
        driver.find_element_by_tag_name('body').send_keys(Keys.END) 
        time.sleep(1)
        scroll -= 1
    
    # 해당 페이지 소스가져오기
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')
    
    # 노래제목 가져오기
    all_title = soup.find_all('a','yt-simple-endpoint style-scope ytd-grid-video-renderer')
    title = [soup.find_all('a','yt-simple-endpoint style-scope ytd-grid-video-renderer')[n].text for n in range(0,len(all_title))]
    
    # url 가져오기
    link = [soup.find_all('a','yt-simple-endpoint style-scope ytd-grid-video-renderer')[n]['href'] for n in range(0,len(all_title))]
    
    tag = []
    comment_all = []
    base_url = 'https://www.youtube.com'
    n = 0
    for i in link :  # 수집된 영상link 하나씩 접속하며 태그 수집
        watch_url = base_url + i
        driver.get(watch_url)
        time.sleep(3) 

        req = driver.page_source
        soup = BeautifulSoup(req, 'html.parser')
        
        tag_list = []
        # container
        tag_list.append(driver.find_element(By.CSS_SELECTOR, '.style-scope ytd-video-primary-info-renderer').text.split('\n')[0])
        # title
        tag_list.append(driver.find_element(By.CSS_SELECTOR,'.style-scope ytd-watch-metadata').text.split('\n')[0])
            
        tag.append(tag_list)
        print(n, ' : ', tag[n])
        n += 1        
        
        # 댓글 수집 페이지 다운
        # 유튜브는 스크롤 내릴때마다 새로운 html 문서가 생김. 
        # 모든 댓글을 수집하기 위해서는 웹페이지의 스크롤을 끝까지 내린 후 크롤링해야함 
        driver.execute_script("window.scrollTo(0, 800)")
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(1)
            new_height = driver.execute_script("return document.documentElement.scrollHeight")

            if new_height == last_height:
                break
            last_height = new_height
            time.sleep(1)

        time.sleep(2)

        # 유튜브 팝업 닫기 - 크롤링시 방해요소로 작용할 수 있음
        try:
            driver.find_element(By.CSS_SELECTOR, "#dismiss-button > a").click()
        except:
            pass

        # 댓글 가져오기
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')

        comments = soup.select("yt-formatted-string#content-text") # 댓글을 모두 불러온다

        comment_list = []

        for i in range(len(comments)):
            temp_comment = comments[i].text # 텍스트만 추출
            temp_comment = temp_comment.replace('\n', '') 
            temp_comment = temp_comment.replace('\t', '')
            temp_comment = temp_comment.replace('    ', '')
            comment_list.append(temp_comment) # 댓글 내용

        comment_all.append(comment_list)  
    
    channel_video_df = pd.DataFrame({'title':title,'link':link,'tag':tag,'comment':comment_all})
    
    print('수집된 영상링크 개수 : ',len(channel_video_df))
    
    return channel_video_df

In [92]:
# 함수호출 및 크롤링하고싶은 채널 주소 입력
channel_video_df = channel_video_crawl('https://www.youtube.com/channel/UC2kPuCfWPDZhRg60gOdOlKg/videos')
channel_video_df

C:\Users\admin\AppData\Local\Temp/ipykernel_23824/2808392149.py:8: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  driver.find_element_by_tag_name('body').send_keys(Keys.END)


0  :  ['', '#우울한날 #슬픈노래 #힘들때듣기좋은노래']
1  :  ['', '#힙합노래모음 #감성힙합 #알앤비']
2  :  ['', '#가족노래모음 #가족노래 #부모님노래']
3  :  ['', '#이별노래모음 #이별플리 #헤어진날']
4  :  ['', '#짝사랑노래 #고백노래 #설레는노래']
5  :  ['', '#여자아이돌노래모음 #걸그룹노래모음 #여돌노래모음']
6  :  ['', '#봄노래모음 #봄에듣기좋은노래 #봄노래']
7  :  ['', '#낮잠잘때듣기좋은노래 #낮잠플리 #나른한노래']
8  :  ['', '#2000년대노래 #2000년대발라드 #발라드노래모음']
9  :  ['', '#아이유노래 #아이유노래모음 #아이유플레이리스트']
10  :  ['', '#인디노래 #인디발라드 #발라드노래']
11  :  ['', '#크리스마스노래 #캐롤모음 #겨울노래']
12  :  ['', '#신스팝 #시티팝 #드라이브음악']
13  :  ['', '#이별노래 #여자발라드 #헤어진날']
14  :  ['', '#겨울노래 #잔잔한겨울노래 #따뜻한노래']
15  :  ['', '#알앤비 #감성알앤비 #가을노래']
16  :  ['', '#가을노래 #옛날노래 #90년대노래']
17  :  ['', '#결혼식축가 #축가 #축가추천']
18  :  ['', '#헤어진날 #이별노래 #이별한날']
19  :  ['', '#새벽감성노래 #새벽에듣기좋은노래 #인디노래']
20  :  ['', '#지칠때듣기좋은노래 #힘나는노래 #우울할때듣기좋은노래']
21  :  ['', '#여행갈때듣기좋은노래 #여행노래 #신나는노래']
22  :  ['', '#90년대여름노래 #여름에듣기좋은노래 #여름노래']
23  :  ['', '#짝사랑 #달달한 #설레는']
24  :  ['', '#힙합노래모음 #국힙노래모음 #감성힙합']
25  :  ['', '#밤에듣기좋은노래 #여름노래 #인디노래']
26  :  ['', '#퇴근할때듣기좋은노래 #퇴근길노래 #지칠때']
27  :  

,title,link,tag,comment
0,"우울한 날, 포근하게 안아주는 위로노래모음 | PLAYLIST",/watch?v=4xsser5_duU,"[, #우울한날 #슬픈노래 #힘들때듣기좋은노래]","[우울한 날, 포근하게 안아주는 위로노래모음 | PLAYLIST 43:22 광고제거..."
1,힙한 하루🤟 적당히 둠칫한 힙합 & 알앤비 노래모음 | PLAYLIST,/watch?v=OiFTo6AkADU,"[, #힙합노래모음 #감성힙합 #알앤비]",[힙한 하루 적당히 둠칫한 힙합노래모음 | PLAYLIST44:38 광고제거\r44...
2,가정의 달👪 부모님 생각나는 가족노래모음 | PALYLIST,/watch?v=a2WXrPQE9UA,"[, #가족노래모음 #가족노래 #부모님노래]",[50:29 광고제거50:29 누르고 다시재생00:01 세정 - 꽃길 (Prod.Z...
3,전남친 전여친 생각나는 이별노래모음 | PLAYLIST,/watch?v=Vh0omXrlS5k,"[, #이별노래모음 #이별플리 #헤어진날]",[전남친 전여친 생각나는 이별노래모음 | PLAYLIST\r53:35 광고제거\r5...
4,짝사랑 말고🙅‍♀️ 짝이 생기게 되는 잔잔한 고백 노래모음 | PLAYLIST,/watch?v=r6hnnDgXPvM,"[, #짝사랑노래 #고백노래 #설레는노래]",[짝사랑 말고 짝이 생기게 되는 잔잔한 고백 노래모음 | PLAYLIST\r 42:...
5,신나는 노동요😎 내적댄스 샘솟는 걸그룹 & 여자아이돌 노래모음 | PLAYLIST,/watch?v=FbwBprHeLrQ,"[, #여자아이돌노래모음 #걸그룹노래모음 #여돌노래모음]","[신나는 노동요, 내적댄스 샘솟는 걸그룹 & 여자 아이돌 노래모음 | PLAYLIS..."
6,[PLAYLIST] 올 봄에는 짝사랑이 성공하길🙏 봄에 듣기 좋은 따뜻한 사랑노래모음,/watch?v=4WNJE9gq030,"[, #봄노래모음 #봄에듣기좋은노래 #봄노래]",[광고제거35:18 누르고 다시재생𝐏𝐋𝐀𝐘𝐋𝐈𝐒𝐓\r00:00 노이마 - 사랑해도 ...
7,"나른한 봄날, 낮잠자며 듣기좋은 인디노래모음😴 | PLAYLIST",/watch?v=vh_JVKkpsL4,"[, #낮잠잘때듣기좋은노래 #낮잠플리 #나른한노래]",[광고제거54:09 누르고 다시재생PLAYLIST00:01 크르르(Krr) - 이사...
8,"[그시절] 나의 오래된 MP3 속, 2000년대 발라드 노래모음 | 𝗣𝗟𝗔𝗬𝗟𝗜𝗦𝗧...",/watch?v=o-QxXe5kfoE,"[, #2000년대노래 #2000년대발라드 #발라드노래모음]",[[광고제거]02:05:50 : 누르고 다시재생[PLAYLIST]00:00:01 윤...
9,낮부터 밤까지 아이유 노래모음 50곡 | IU Playlist Best 50 Son...,/watch?v=3FoJqr5pIS4,"[, #아이유노래 #아이유노래모음 #아이유플레이리스트]",[광고제거03:06:37 누르고 다시재생𝐏𝐋𝐀𝐘𝐋𝐈𝐒𝐓\r00:00:00 아이유 -...


#### 수집된 DF -> CSV 로 저장

In [93]:
channel_video_df.to_csv('./crawl_data/youtube_channel_1.csv',  encoding='utf8')

In [95]:
channel_video_df['comment'][1]

["힙한 하루 적당히 둠칫한 힙합노래모음 | PLAYLIST44:38 광고제거\r44:38 누르고 다시재생\r 00:01 SINDOSI:22 - Domythang\r00:01 SINDOSI:22 - Domythang\r 02:48 LIlBOI & 원슈타인 - Friends\r02:48 LIlBOI & 원슈타인 - Friends\r 05:58 DPR LIVE - Action! (feat.GRAY)\r05:58 DPR LIVE - Action! (feat.GRAY)\r 09:10 DPR LIVE x BEENZINO x HWASA - Hula Hoops\r09:10 DPR LIVE x BEENZINO x HWASA - Hula Hoops\r 12:33 딘딘(DINDIN) - 돼버릴거야 (feat.휘인) (Prod.기리보이)\r12:33 딘딘(DINDIN) - 돼버릴거야 (feat.휘인) (Prod.기리보이)\r 16:27 K.vsh(캐시) - Better (Feat.기리보이)\r16:27 K.vsh(캐시) - Better (Feat.기리보이)\r 19:29 Leellamarz(릴러말즈) & TOIL(토일) - Long Way (Feat.1day,Urb fisher)\r19:29 Leellamarz(릴러말즈) & TOIL(토일) - Long Way (Feat.1day,Urb fisher)\r 22:33 Sik-K(식케이) - h1ghr gang (Prod.BOYCOLD)\r22:33 Sik-K(식케이) - h1ghr gang (Prod.BOYCOLD)\r 27:25 Beenzino(빈지노) - Nike Shoes (Feat.다이나믹듀오)\r27:25 Beenzino(빈지노) - Nike Shoes (Feat.다이나믹듀오)\r 31:57 BE'O(비오) - Counting Stars (Feat. Beenzino)\r31:57 BE'O(비오) - Counting Stars (Feat. Beenzino)\r 34:28 BE'O(비오) - 리무진 (Feat.MINO) (Pr

#### -------------------------------------- 테스트 --------------------------------------

In [75]:
driver.get('https://www.youtube.com/watch?v=4qh8D_aPX5A')

driver.execute_script("window.scrollTo(0, 800)")

# 페이지 다운
# 유튜브는 스크롤 내릴때마다 새로운 html 문서가 생김. 
# 모든 댓글을 수집하기 위해서는 웹페이지의 스크롤을 끝까지 내린 후 크롤링해야함 
last_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")

    if new_height == last_height:
        break
    last_height = new_height
    time.sleep(1)

time.sleep(2)

# 유튜브 팝업 닫기 - 크롤링시 방해요소로 작용할 수 있음
try:
    driver.find_element(By.CSS_SELECTOR, "#dismiss-button > a").click()
except:
    pass
    
# 댓글 가져오기
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')

# id_list = soup.select("div#header-author > h3 > #author-text > span")
comments = soup.select("yt-formatted-string#content-text") # 댓글을 모두 불러온다

# id_final = []
comment_list = []
comment_all = []

re_coms = []
for i in range(len(comments)):
#     temp_id = id_list[i].text
#     temp_id = temp_id.replace('\n', '')
#     temp_id = temp_id.replace('\t', '')
#     temp_id = temp_id.replace('    ', '')
#     id_final.append(temp_id) # 댓글 작성자

    temp_comment = comments[i].text # 텍스트만 추출
    temp_comment = temp_comment.replace('\n', '') 
    temp_comment = temp_comment.replace('\t', '')
    temp_comment = temp_comment.replace('    ', '')
    comment_list.append(temp_comment) # 댓글 내용

comment_all.append(comment_list)
print(comment_all)
# 저장
# pd_data = {"아이디" : id_final , "댓글 내용" : comment_final}
# youtube_pd = pd.DataFrame(pd_data)
# youtube_pd

TypeError: 'WebElement' object is not iterable

In [72]:
len(comment_all)
len(comment_all[0])
re_coms

180

In [213]:
#  DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
# 에러발생으로 검색해보니 find_element(BY.CSS_SELECTOR) 이 구문으로 사용하라고 함.. 다시 수정
from selenium.webdriver.common.by import By

req = driver.page_source
# req
soup = BeautifulSoup(req, 'html.parser')

tag = driver.find_element(By.CSS_SELECTOR,'.style-scope ytd-watch-metadata').text.split('\n')[0]
tag

'#걸그룹 #여돌 #아이돌'